In [58]:
import requests
from bs4 import BeautifulSoup as soup
from collections import Counter
import lxml
import pandas as pd

import chromedriver_binary
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time 
from time import sleep 

In [ ]:
def process_zillset(html):
    """
    This function processes the zillow html for the differnt urls and returns the items of interest
    """
    for home in html:
        home.prettify()
        home.name, home.attrs, home['class']

        #address
        try:
            zaddress = home.find_all('address')
            address = zaddress[0].text.strip()
            zill_dict['address'].append(address)
        except IndexError:
            zill_dict['address'].append('NaN')
            
        
        #listing brokerage and agent 
        try:
            zlisting = home.find_all('p', {'class':'list-card-extra-info'})
            listingby = zlisting[0].text.strip()
            zill_dict['listingby'].append(listingby)
        except IndexError:
            zill_dict['listingby'].append('NaN')
            
        
        #price
        try:
            zprice = home.find_all('div', {'class':'list-card-price'})
            price = zprice[0].text.strip()
            zill_dict['price'].append(price)
        except IndexError:
            zill_dict['price'].append('NaN')
            
        
        #beds
        try:
            zbed = home.find_all('li', {'class':''})
            beds = zbed[0].text.strip()
            zill_dict['beds'].append(beds)
        except IndexError:
            zill_dict['beds'].append('NaN')
            
        
        #bathrooms
        try:
            zbath = home.find_all('li', {'class':''})
            baths = zbath[1].text.strip()
            zill_dict['baths'].append(baths)
        except IndexError:
            zill_dict['baths'].append('NaN')
            
        
        #sqft
        try:
            zsqft = home.find_all('li', {'class':''})
            sqft = zsqft[2].text.strip()
            zill_dict['sqft'].append(sqft)
        except IndexError:
            zill_dict['sqft'].append('NaN')
            
            
        #type
        try:
            ztype = home.find_all('li', {'class':'list-card-statusText'})
            types = ztype[0].text.strip()
            zill_dict['building_type'].append(types)
        except IndexError:
            zill_dict['building_type'].append('NaN')
            
    return zill_dict
    

In [59]:
# make _urls returns a list of Zillow urls, incrementing the page number until it reaches the max_page 
def make_urls(max_page):
    url_list =[]
    for pageNumber in range(1,max_page+1):
        #url = 'https://www.zillow.com/san-francisco-ca/' + str(pageNumber) + '_p?'
        url = 'https://www.zillow.com/san-francisco-ca/rentals/'+ str(pageNumber) + '_p?'
        url_list += [url]
    return url_list 

In [89]:
# we give the max_page and call make_urls to make a list of urls that we will use 
max_page = 2
zillow_urls = make_urls(max_page)
print(zillow_urls)

['https://www.zillow.com/san-francisco-ca/rentals/1_p?', 'https://www.zillow.com/san-francisco-ca/rentals/2_p?']


In [ ]:
zill_dict = {'address':[], 'listingby':[], 'price':[],'beds':[], 'baths':[],'sqft':[], 'building_type':[], 'price/sqft':[]}
for url in zillow_urls:

    driver = webdriver.Chrome()
    page = driver.get(url)
    time.sleep(20)
    zillhtml= driver.page_source

    #making into soup object
    zill = soup(zillhtml,'lxml') 
    zill_content = zill.find_all('div', attrs ={'class':'list-card-info'})
    print(url)
    zill_count = 1
    for zills in zill_content:
        zill_count+=1
    dict=process_zillset(zill_content)
    df = pd.DataFrame.from_dict(dict, orient='index').transpose()

In [230]:
zillow_fs = pd.read_csv("ZillowData.csv", index_col=[0])

zillow_fs.shape

(800, 7)

In [196]:
# removing unwanted parts in column values
zillow_fs['sqft'] = zillow_fs['sqft'].str.rstrip('sqft')
zillow_fs['sqft'] = zillow_fs['sqft'].str.replace(',', '')
zillow_fs['sqft'] = zillow_fs['sqft'].str.replace('--', '0')
zillow_fs['baths']= zillow_fs['baths'].str.rstrip('ba')
zillow_fs['beds'] = zillow_fs['beds'].str.rstrip('bds')
zillow_fs['price'] = zillow_fs['price'].str.replace('$', '')
zillow_fs['price'] = zillow_fs['price'].str.replace(',', '')
zillow_fs['price'] = zillow_fs['price'].str.rstrip('+')

In [197]:
# splitting up string values in columns
zillow_fs['zip']= zillow_fs['address'].str.split().str[-1]
zillow_fs['address'] = zillow_fs['address'].str.split(',',expand =True)[0]
zillow_fs['agent'] = zillow_fs['listingby'].str.split(',', expand=True)[1]
zillow_fs['brokerage'] = zillow_fs['listingby'].str.split(',', expand=True)[0]
zillow_fs = zillow_fs.drop(columns =['listingby'])

In [200]:
# dividing price and sqft column to create price per sqft
zillow_fs=zillow_fs.fillna(0)
zillow_fs[['price', 'sqft']] = zillow_fs[['price', 'sqft']].apply(pd.to_numeric)
zillow_fs['price/sqft'] = zillow_fs['price'].astype(int) / zillow_fs['sqft'].astype(int)

In [227]:
zillow_fs = zillow_fs[['address', 'zip', 'building_type', 'price', 'beds', 'baths', 'sqft', 'price/sqft', 'agent', 'brokerage']]
zillow_fs.head()

,address,zip,building_type,price,beds,baths,sqft,price/sqft,agent,brokerage
0,710 London St,94112,- House for sale,1498000,4,3,2047,731.802638,Darin J. Holwitz,COMPASS
1,655 27th St,94131,- House for sale,2195000,3,3,1791,1225.572306,Dale T. Boutiette,COMPASS SF
2,270 Sadowa St,94112,- House for sale,1049000,3,2,1264,829.905063,Cynthia M. Pagan,PACIFIC EDGE
3,437 Valley St,94131,- House for sale,2000000,3,2,1949,1026.167265,Allison Fortini Crawford,SOTHEBY'S INTERNATIONAL REALTY
4,585 Rockdale Dr,94127,- House for sale,1295000,2,1,1152,1124.131944,Edward O'Connell,CORCORAN GLOBAL LIVING


In [226]:
zillow_fs['brokerage'].value_counts()

COMPASS                           156
0                                 111
VANGUARD PROPERTIES                61
SOTHEBY'S INTERNATIONAL REALTY     58
COMPASS SF                         48
                                 ... 
HELM REAL ESTATE                    1
JENNY HE REALTY ASSOCIATES          1
SHANNON MOLLOY                      1
WEST COAST PROPERTY MGT. CO.        1
PINNACLE PROPERTIES                 1
Name: brokerage, Length: 147, dtype: int64

In [219]:
zillow_fs.to_csv('for_sale.csv')

In [221]:
zillow_fr = pd.read_csv("ZillowRent.csv", index_col=[0])
zillow_fr.head()

,address,listingby,price,beds,baths,sqft,building_type,price/sqft
0,"956 Valencia St. | 956 Valencia St, San Franci...",NaN,"$4,595+ 4 bds",NaN,NaN,NaN,NaN,NaN
1,"333 Main St UNIT 4J, San Francisco, CA 94105",NaN,"$3,650/mo",1 bd,1 ba,815 sqft,- Apartment for rent,NaN
2,"1271 10th Ave, San Francisco, CA 94122",NaN,"$4,900/mo",3 bds,1.5 ba,"1,675 sqft",- Apartment for rent,NaN
3,"2027 Market St APT 9, San Francisco, CA 94114",NaN,"$4,200/mo",2 bds,2 ba,"1,032 sqft",- Condo for rent,NaN
4,"161 Broderick St #3, San Francisco, CA 94117",NaN,"$2,795/mo",1 bd,1 ba,910 sqft,- Apartment for rent,NaN


In [225]:
#zillow_fr['brokerage'].value_counts()

In [216]:
# removing unwanted string parts in column
zillow_fr = zillow_fr.drop(columns =['listingby'])
zillow_fr = zillow_fr.rename(columns={'price':'rent/month'}, inplace=True)
zillow_fr['price'] = zillow_fr['price'].str.replace('$', '')
zillow_fr['price'] = zillow_fr['price'].str.replace(',', '')
zillow_fr['price'] = zillow_fr['price'].str.replace('/mo', '')
zillow_fr['beds'] = zillow_fr['beds'].str.rstrip('bds')
zillow_fr['beds'] = zillow_fr['beds'].str.rstrip('bd')
zillow_fr['baths'] = zillow_fr['baths'].str.rstrip('ba')
zillow_fr['sqft'] = zillow_fr['sqft'].str.rstrip('sqft')
zillow_fr['sqft'] = zillow_fr['sqft'].str.replace(',', '')
zillow_fr

TypeError: 'NoneType' object is not subscriptable

In [ ]:
zi